# Notion AI Agent

Code authored by: Shaw Talebi

### imports

In [1]:
from agents.mcp.server import MCPServerStdio, MCPServer
from agents import Agent, Runner, gen_trace_id, trace, ModelSettings
from openai.types.responses import ResponseTextDeltaEvent
import os
import asyncio

In [2]:
from dotenv import load_dotenv
load_dotenv()

# import notion api key
notion_key = os.getenv("NOTION_API_KEY")

### explore MCP server tools

In [3]:
async with MCPServerStdio(
    params={
        "command": "npx",
        "args": ["-y", "@notionhq/notion-mcp-server"],
        "env": {
        "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
        }
    }
) as server:
    tool_list = await server.list_tools()

In [4]:
toolname_list = [tool.name for tool in tool_list]
print(toolname_list)

['API-get-user', 'API-get-users', 'API-get-self', 'API-post-database-query', 'API-post-search', 'API-get-block-children', 'API-patch-block-children', 'API-retrieve-a-block', 'API-update-a-block', 'API-delete-a-block', 'API-retrieve-a-page', 'API-patch-page', 'API-post-page', 'API-create-a-database', 'API-update-a-database', 'API-retrieve-a-database', 'API-retrieve-a-page-property', 'API-retrieve-a-comment', 'API-create-a-comment']


In [5]:
[tool.description for tool in tool_list]

['Retrieve a user\nError Responses:\n400: 400',
 'List all users\nError Responses:\n400: 400',
 "Retrieve your token's bot user",
 'Query a database',
 'Search by title',
 'Retrieve block children',
 'Append block children',
 'Retrieve a block',
 'Update a block',
 'Delete a block',
 'Retrieve a page',
 'Update page properties',
 'Create a page',
 'Create a database',
 'Update a database',
 'Retrieve a database',
 'Retrieve a page property item',
 'Retrieve comments',
 'Create comment']

### creat main with MCP server

In [6]:
async def main():
    
    async with MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "@notionhq/notion-mcp-server"],
            "env": {
            "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
            }
        }
    ) as server:
        trace_id = gen_trace_id()
        with trace(workflow_name="Notion Agent Example", trace_id=trace_id):
            print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}\n")
            await run(server)
        
        tools = await server.list_tools()

### create run() with agent

In [7]:
with open('instructions.md', 'r', encoding='utf-8') as file:
    instructions = file.read()

In [8]:
# async def run(mcp_server: MCPServer):
#     agent = Agent(
#         name="Notion Agent",
#         model="gpt-4.1-2025-04-14",
#         instructions=instructions,
#         mcp_servers=[mcp_server],
#     )
#     ModelSettings.tool_choice = "required"

#     # get user input
#     message = input("\nUser: ").strip()
    
#     # run agent
#     result = await Runner.run(starting_agent=agent, input=message)
#     print(result.final_output)

In [9]:
async def run(mcp_server: MCPServer):
    agent = Agent(
        name="Notion Agent",
        model="gpt-4.1-2025-04-14",
        instructions=instructions,
        mcp_servers=[mcp_server],
    )
    ModelSettings.tool_choice = "required"
    
    input_items = []

    print("=== Notion AI Agent ===")
    print("Type 'exit' to end the conversation")

    while True:
        # Get user input
        user_input = input("\nUser: ").strip()
        input_items.append({"content": user_input, "role": "user"})
        
        # Check for exit command
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nGoodbye!")
            break
            
        if not user_input:
            continue

        print("\nAgent: ", end="", flush=True)
        result = Runner.run_streamed(
            agent,
            input=input_items,
        )
        
        async for event in result.stream_events():
            # We'll ignore the raw responses event deltas for text
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
            elif event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    print(f"\n-- Calling Tool: {event.item.raw_item.name}...")
                elif event.item.type == "tool_call_output_item":
                    print("-- Tool call completed.")
                elif event.item.type == "message_output_item":
                    input_items.append({"content": f"{event.item.raw_item.content[0].text}", "role": "assistant"})
                else:
                    pass  # Ignore other event types

        print("\n")  # Add a newline after each response

# ... existing code ...

### run agent

In [10]:
# # to run in a .py script use
# if __name__ == "__main__":
#     asyncio.run(main())

In [11]:
#message = "Write a draft for the 'AI won’t make you rich' LinkedIn post idea. Then add the final post to the page contents."
await main()

View trace: https://platform.openai.com/traces/trace?trace_id=trace_e03e30253fa04e88953316353b538a92

=== Notion AI Agent ===
Type 'exit' to end the conversation



User:  Search for post ideas on AI consulting



Agent: 
-- Calling Tool: API-post-search...
-- Tool call completed.
Here are some recent post ideas on AI consulting found in the Notion workspace:

1. **AI won’t make you rich (solving problems will)**
   - Theme: Focusing on problems over hype in AI consulting.
   - URL: (Internal Notion link)

2. **Quitting Consulting**
   - Theme: Why Shaw is quitting AI consulting to focus on YouTube (focus, results from YouTube, and the benefits of deep specialization).
   - URL: (Internal Notion link)

3. **How to find (in-demand) AI freelancing service using ChatGPT**
   - Theme: Using ChatGPT to research and validate AI consulting and freelancing service ideas.
   - URL: (Internal Notion link)

4. **My 2025 AI Product Development Tech Stack**
   - Theme: Tools and technologies for AI consulting/product development in 2025.
   - URL: (Internal Notion link)

5. **5 AI projects…**
   - Theme: Project ideas for building portfolio as an AI consultant/freelancer.
   - URL: (Internal Notion link)

I


User:  Let's flesh out idea 3 based on the content



Agent: 
-- Calling Tool: API-post-search...
-- Tool call completed.

-- Calling Tool: API-get-block-children...
-- Tool call completed.
Step 1: Purpose & Audience
- Purpose: Educate and provide a practical, clearly outlined method for finding in-demand AI freelancing/consulting service ideas using ChatGPT.
- Audience: Aspiring and early-stage AI freelancers, consultants, and technical professionals considering a freelance path.

Step 2: Post Wireframe

[Hook: Direct, practical, curiosity-driven—maybe focus on "the fastest way to validate what AI services are actually getting paid for"]

[Meat/Body]
- Stepwise guide showing how to use actual job postings, plus ChatGPT, to identify in-demand, well-paying services/niches.
- Brief explanation for each step (not just a list).
- Framing: This is the actual process, not just theory.

[CTA]
- Ask for additional methods, or for readers to share their own favorite tools for service validation.

Step 3: Body (“meat”)

The fastest way I know to v

...


User:  You can add content not replace it



Agent: 
-- Calling Tool: API-post-search...
-- Tool call completed.

-- Calling Tool: API-get-block-children...
-- Tool call completed.

-- Calling Tool: API-patch-block-children...
-- Tool call completed.
The final LinkedIn post has been successfully added as a new content block at the end of the Notion page "How to find (in-demand) AI freelancing service using ChatGPT." Existing content on the page was left unchanged. 

If you want to review, revise, or add more content, just let me know!




User:  exit



Goodbye!



User:  Return post ideas about business and entrepreneurship
